In [68]:
# Run this cell to run all functions in this file

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import datetime as dt


In [8]:
# Lists below to use in functions below to pull data for
# Jan - May in the years of 2018, 2019, 2020
# Run cell to use lists later.

# MTA weeks to pull to create 2020 files
week_nums_2020 = [200104, 200111, 200118, 200125, 200201, 
                  200208, 200215, 200222, 200229, 200307, 
                  200314, 200321, 200328, 200404, 200411, 
                  200418, 200425, 200502, 200509, 200516, 
                  200523, 200530, 200606, 200613, 200620, 
                  200627]

# MTA weeks to pull to create 2019 files
week_nums_2019 = [190105, 190112, 190119, 190126, 190202, 
                  190209, 190216, 190223, 190302, 190309, 
                  190316, 190323, 190330, 190406, 190413, 
                  190420, 190427, 190504, 190511, 190518, 
                  190525, 190601, 190608, 190615, 190622, 
                  190629]

# MTA weeks to pull to create 2018 files
week_nums_2018 = [180106, 180113, 180120, 180127, 180203, 
                  180210, 180217, 180224, 180303, 180310, 
                  180317, 180324, 180331, 180407, 180414, 
                  180421, 180428, 180505, 180512, 180519, 
                  180526, 180602, 180609, 180616, 180623, 
                  180630]

In [ ]:
# Function that will do bulk-pull of MTA data for 
# multiple weeks.  Run cell for use later in 'master'
# function.

def get_data(week_nums):
    url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt"
    dfs = []
    for week_num in week_nums:
        file_url = url.format(week_num)
        dfs.append(pd.read_csv(file_url))
    return pd.concat(dfs)

## SMALL FUNCTIONS NEEDED MASTER FUNCTIONS

In [ ]:
# Run this cell -- must be done before running 
# functions below. This function and one below
# (get_day_exit_counts()) will be used in 'master'
# functions below

def get_day_entry_counts(row, max_counter):
    counter = row["ENTRIES"] - row["PREV_EXITS"]
    if counter < 0:
        counter = -counter
    if counter > max_counter:
        counter = min(row["ENTRIES"], row["PREV_ENTRIES"])
    if counter > max_counter:
        return 0
    return counter


In [ ]:
# As noted in previous cell, run this function before
# running 'master' functions

def get_day_exit_counts(row, max_counter):
    counter = row["EXITS"] - row["PREV_EXITS"]
    if counter < 0:
        counter = -counter
    if counter > max_counter:
        counter = min(row["EXITS"], row["PREV_EXITS"])
    if counter > max_counter:
        return 0
    return counter

## MASTER FUNCTION FOR STATION-ONLY DF

In [86]:
# 'Master' function to get data for target stations
# that are not line-dependent


def get_yearly_data_station_only(list_of_weeks):
    '''
    Input: A list of dates that match the end of the 
    MTA raw data files that you want to pull from in the 
    format YYMMDD, such as "200104."
    
    Output: will be a dataframe that lists only data
    for the stations: Times Sq, Grand Central, Lexington,
    34 St. Herald, St.Union, 59 St Columbus, and Penn
    Station.  For those stations will also see:
    1. Date
    2. Daily Entries
    3. Daily Exits
    4. Week Of date
    5. Total Traffic (Entries + Exits)
    '''
    df = get_data(week_nums)
    
    # create df with only stations focusing on
    df_stations = df[(df.STATION == 'TIMES SQ-42 ST')
                     |(df.STATION == 'GRD CNTRL-42 ST')
                     |(df.STATION == 'LEXINGTON AV/53')
                     |(df.STATION == '34 ST-HERALD SQ')
                     |(df.STATION == '14 ST-UNION SQ')
                     |(df.STATION == '59 ST COLUMBUS')
                     |(df.STATION == '34 ST-PENN STA')]


    df_stations.rename(columns={df_stations.columns[10]: 'EXITS'}
                       , inplace=True)

    
    # find first entry for entries and exits
    day_counts_df = df_stations.groupby(['C/A','UNIT','SCP','STATION','DATE']
                                    ,as_index=False)[['ENTRIES','EXITS']].first()
    
    # get the ENTRY daily summary data of each turnstile
    day_counts_df[['PREV_ENTRY_DATE', 'PREV_ENTRIES']] = (day_counts_df
                                                          .groupby(['C/A', 'UNIT', 'SCP', 'STATION'])['DATE', 'ENTRIES']
                                                          .apply(lambda grp: grp.shift(1)))
   
    # get the EXIT daily summary data of each turnstile
    day_counts_df[['PREV_EXIT_DATE', 'PREV_EXITS']] = (day_counts_df
                                                       .groupby(['C/A', 'UNIT', 'SCP', 'STATION'])['DATE', 'EXITS']
                                                       .apply(lambda grp: grp.shift(1)))
    
    
    # drop nan where applicable in new previous date columns for entries & exits
    day_counts_df.dropna(subset=['PREV_ENTRY_DATE'], inplace=True)
    day_counts_df.dropna(subset=['PREV_EXIT_DATE'], inplace=True)
    

    #create new columns for daily entries and daily exits
    day_counts_df["DAILY_ENTRIES"] = (day_counts_df
                                            .apply(get_day_entry_counts, axis=1, max_counter=100_000))
    
    day_counts_df["DAILY_EXITS"] = (day_counts_df
                                    .apply(get_day_exit_counts, axis=1, max_counter=100_000))
    
    
    # set datetime data
    day_counts_df['DATE'] = pd.to_datetime(day_counts_df['DATE'])
    
    
    # get the daily summary data of station
    daily_sum_stations = day_counts_df.groupby(
        ['STATION','DATE'])[['DAILY_ENTRIES','DAILY_EXITS']].sum().reset_index()
    
    # get the fist day of the week & place in new 'WeekOf' column
    daily_sum_stations['weekday'] = daily_sum_stations['DATE'].dt.dayofweek
    daily_sum_stations['WeekOf'] = daily_sum_stations['DATE']
    daily_sum_stations.loc[daily_sum_stations.weekday != 0, 'WeekOf'] = ''
    daily_sum_stations = daily_sum_stations.replace('',np.nan).ffill()
    daily_sum_stations['WeekOf'] = pd.to_datetime(daily_sum_stations['WeekOf']).dt.date
    
    # get total Traffic (= entries + exits) & place in new 'TotalTraffic' column
    daily_sum_stations['TotalTraffic'] = (daily_sum_stations['DAILY_ENTRIES'] 
                                          + daily_sum_stations['DAILY_EXITS'])
    
    
    return daily_sum_stations

# << BELOW FUNCTIONS NEED TO BE UPDATED TO INCLUDE NEW WAY TO GET DAILY NUMBERS >>

# NO GROUPBY LINENAME

### FULTON ST

In [17]:
def get_yearly_data_station_and_line(list_of_weeks):
    '''
    Input: A list of dates that match the end of the 
    MTA raw data files that you want to pull from in the 
    format YYMMDD, such as "200104."
    
    Output: will be a dataframe that lists only data
    for the Fulton St's lines: 'ACJZ2345'
    and '2345ACJZ'.  Data for two lines will be 
    combined under Fulton as one.  Will also see:
    1. Date
    2. Daily Entries
    3. Daily Exits
    4. Week Of date
    5. Total Traffic (Entries + Exits)
    '''
    df = get_data(week_nums)
    
    # create df with only stations focusing on
    df_stations = df[(df.STATION == "FULTON ST")]

    # clean data
    df_stations['DATE'] = pd.to_datetime(df_stations['DATE'])
    df_stations['TIME'] = pd.to_datetime(df_stations['TIME'])
    df_stations.rename(columns={df_stations.columns[10]: 'EXITS'}
                            , inplace=True)

    # get the daily summary data of each turnstile
    daily_sum_stations = df_stations.groupby(
        ['C/A', 'UNIT', 'SCP', 'LINENAME', 'STATION','DATE']).agg(
        {'ENTRIES': lambda ENTIRES : ENTIRES.max() - ENTIRES.min(),
         'EXITS': lambda EXITS: EXITS.max() - EXITS.min()}).reset_index()
    
    # get the lines we need for "FULTON ST"
    daily_sum_stations = daily_sum_stations[(daily_sum_stations.LINENAME == 'ACJZ2345')
                                     |(daily_sum_stations.LINENAME == '2345ACJZ')]
    
    # get the daily summary data of station
    daily_sum_stations = daily_sum_stations.groupby(
        ['STATION','DATE'])[['ENTRIES','EXITS']].sum().reset_index()
    
    # get the fist day of the week & place in new 'WeekOf' column
    daily_sum_stations['weekday'] = daily_sum_stations['DATE'].dt.dayofweek
    daily_sum_stations['WeekOf'] = daily_sum_stations['DATE']
    daily_sum_stations.loc[daily_sum_stations.weekday != 0, 'WeekOf'] = ''
    daily_sum_stations = daily_sum_stations.replace('',np.nan).ffill()
    daily_sum_stations['WeekOf'] = pd.to_datetime(daily_sum_stations['WeekOf']).dt.date
    
    # get total Traffic (= entries + exits) & place in new 'TotalTraffic' column
    daily_sum_stations['TotalTraffic'] = daily_sum_stations['ENTRIES'] + daily_sum_stations['EXITS']
    
    
    return daily_sum_stations

### CHAMBERS ST

In [18]:
def chambers_yearly_data(week_nums):
    '''
    Input: A list of dates that match the end of the 
    MTA raw data files that you want to pull from in the 
    format YYMMDD, such as "200104."
    
    Output: will be a dataframe that lists only data
    for the Chambers' lines: 'JZ456'and 'ACE23'.  Data 
    for two lines will be combined under Fulton as one.  
    Will also see:
    1. Date
    2. Daily Entries
    3. Daily Exits
    4. Week Of date
    5. Total Traffic (Entries + Exits)
    '''
    df = get_data(week_nums)
    
    # create df with only stations focusing on
    df_stations = df[(df.STATION == "CHAMBERS ST")]

    # clean data
    df_stations['DATE'] = pd.to_datetime(df_stations['DATE'])
    df_stations['TIME'] = pd.to_datetime(df_stations['TIME'])
    df_stations.rename(columns={df_stations.columns[10]: 'EXITS'}
                            , inplace=True)

    # get the daily summary data of each turnstile
    daily_sum_stations = df_stations.groupby(
        ['C/A', 'UNIT', 'SCP', 'LINENAME', 'STATION','DATE']).agg(
        {'ENTRIES': lambda ENTIRES : ENTIRES.max() - ENTIRES.min(),
         'EXITS': lambda EXITS: EXITS.max() - EXITS.min()}).reset_index()
    
    # get the lines we need for "CHAMBERS ST"
    daily_sum_stations = daily_sum_stations[(daily_sum_stations.LINENAME == 'JZ456')
                                     |(daily_sum_stations.LINENAME == 'ACE23')]
    
    # get the daily summary data of station
    daily_sum_stations = daily_sum_stations.groupby(
        ['STATION','DATE'])[['ENTRIES','EXITS']].sum().reset_index()
    
    # get the fist day of the week & place in new 'WeekOf' column
    daily_sum_stations['weekday'] = daily_sum_stations['DATE'].dt.dayofweek
    daily_sum_stations['WeekOf'] = daily_sum_stations['DATE']
    daily_sum_stations.loc[daily_sum_stations.weekday != 0, 'WeekOf'] = ''
    daily_sum_stations = daily_sum_stations.replace('',np.nan).ffill()
    daily_sum_stations['WeekOf'] = pd.to_datetime(daily_sum_stations['WeekOf']).dt.date
    
    # get total Traffic (= entries + exits) & place in new 'TotalTraffic' column
    daily_sum_stations['TotalTraffic'] = daily_sum_stations['ENTRIES'] + daily_sum_stations['EXITS']
    
    
    return daily_sum_stations

# BY YEAR

### <<EVERYTHING BELOW NEEDS TO BE CLEANED UP!!>>

In [42]:
df_2020 = pd.concat([stations_2020,fulton_2020,chambers_2020])
df_2020['Year'] = '2020'
df_2019 = pd.concat([stations_2019,fulton_2019,chambers_2019])
df_2019['Year'] = '2019'
df_2018 = pd.concat([stations_2018,fulton_2018,chambers_2018])
df_2018['Year'] = '2018'

In [43]:
df_2020

,STATION,DATE,ENTRIES,EXITS,weekday,WeekOf,TotalTraffic,Year
0,14 ST-UNION SQ,2019-12-28,45068,52230,5,NaT,97298,2020
1,14 ST-UNION SQ,2019-12-29,36281,42165,6,NaT,78446,2020
2,14 ST-UNION SQ,2019-12-30,57899,65968,0,2019-12-30,123867,2020
3,14 ST-UNION SQ,2019-12-31,55468,59503,1,2019-12-30,114971,2020
4,14 ST-UNION SQ,2020-01-01,30936,33736,2,2019-12-30,64672,2020
...,...,...,...,...,...,...,...,...
177,CHAMBERS ST,2020-06-22,3611,3744,0,2020-06-22,7355,2020
178,CHAMBERS ST,2020-06-23,4036,4288,1,2020-06-22,8324,2020
179,CHAMBERS ST,2020-06-24,4117,4352,2,2020-06-22,8469,2020
180,CHAMBERS ST,2020-06-25,4270,4558,3,2020-06-22,8828,2020


In [44]:
df_2019

,STATION,DATE,ENTRIES,EXITS,weekday,WeekOf,TotalTraffic,Year
0,14 ST-UNION SQ,2018-12-29,53095,58799,5,NaT,111894,2019
1,14 ST-UNION SQ,2018-12-30,42082,47900,6,NaT,89982,2019
2,14 ST-UNION SQ,2018-12-31,52430,51727,0,2018-12-31,104157,2019
3,14 ST-UNION SQ,2019-01-01,31859,32006,1,2018-12-31,63865,2019
4,14 ST-UNION SQ,2019-01-02,75240,72300,2,2018-12-31,147540,2019
...,...,...,...,...,...,...,...,...
177,CHAMBERS ST,2019-06-24,35831,29920,0,2019-06-24,65751,2019
178,CHAMBERS ST,2019-06-25,36933,30282,1,2019-06-24,67215,2019
179,CHAMBERS ST,2019-06-26,37296,31711,2,2019-06-24,69007,2019
180,CHAMBERS ST,2019-06-27,37566,32116,3,2019-06-24,69682,2019


In [45]:
df_2018

,STATION,DATE,ENTRIES,EXITS,weekday,WeekOf,TotalTraffic,Year
0,14 ST-UNION SQ,2017-12-30,46274,52463,5,NaT,98737,2018
1,14 ST-UNION SQ,2017-12-31,37828,39833,6,NaT,77661,2018
2,14 ST-UNION SQ,2018-01-01,29589,30736,0,2018-01-01,60325,2018
3,14 ST-UNION SQ,2018-01-02,72370,71897,1,2018-01-01,144267,2018
4,14 ST-UNION SQ,2018-01-03,81922,79888,2,2018-01-01,161810,2018
...,...,...,...,...,...,...,...,...
177,CHAMBERS ST,2018-06-25,36881,29621,0,2018-06-25,66502,2018
178,CHAMBERS ST,2018-06-26,39684,32078,1,2018-06-25,71762,2018
179,CHAMBERS ST,2018-06-27,38758,30479,2,2018-06-25,69237,2018
180,CHAMBERS ST,2018-06-28,37421,30056,3,2018-06-25,67477,2018


# THE FINAL DF!

In [46]:
df_all = pd.concat([df_2020, df_2019, df_2018])

In [47]:
df_all

,STATION,DATE,ENTRIES,EXITS,weekday,WeekOf,TotalTraffic,Year
0,14 ST-UNION SQ,2019-12-28,45068,52230,5,NaT,97298,2020
1,14 ST-UNION SQ,2019-12-29,36281,42165,6,NaT,78446,2020
2,14 ST-UNION SQ,2019-12-30,57899,65968,0,2019-12-30,123867,2020
3,14 ST-UNION SQ,2019-12-31,55468,59503,1,2019-12-30,114971,2020
4,14 ST-UNION SQ,2020-01-01,30936,33736,2,2019-12-30,64672,2020
...,...,...,...,...,...,...,...,...
177,CHAMBERS ST,2018-06-25,36881,29621,0,2018-06-25,66502,2018
178,CHAMBERS ST,2018-06-26,39684,32078,1,2018-06-25,71762,2018
179,CHAMBERS ST,2018-06-27,38758,30479,2,2018-06-25,69237,2018
180,CHAMBERS ST,2018-06-28,37421,30056,3,2018-06-25,67477,2018
